<a href="https://colab.research.google.com/github/FTPGitHub/TDG/blob/main/notebooks/kprototypes(version_simple).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kmodes

In [18]:
import pathlib
import pandas as pd
import numpy as np
import seaborn as sns
import plotnine
import time

from pathlib import Path
from sklearn.metrics import silhouette_score
from kmodes.kprototypes import KPrototypes
from plotnine import *

In [20]:
sourcepath = 'https://raw.githubusercontent.com/FTPGitHub/TDG/main/datos'

In [21]:
# Lee dataset estandarizado de transacciones
dfNoSupervisado1 = pd.read_csv("{}/transacciones.csv".format(sourcepath))

In [22]:
dfNoSupervisado1.AISLE = dfNoSupervisado1.AISLE.astype(str)

In [23]:
dfNoSupervisado1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16239 entries, 0 to 16238
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AISLE     16239 non-null  object 
 1   IG        16239 non-null  object 
 2   HTS       16239 non-null  object 
 3   PLANNING  16239 non-null  object 
 4   ABC       16239 non-null  object 
 5   UOM       16239 non-null  object 
 6   CAT       16239 non-null  object 
 7   COSTX     16239 non-null  float64
 8   QTYVAR    16239 non-null  float64
dtypes: float64(2), object(7)
memory usage: 1.1+ MB


In [ ]:
dfNoSupervisado1.head()

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029976,-0.053250
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092472,-0.022999
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266130,0.004373
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.025112,-0.060323
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077982,-0.062534


In [ ]:
dfNoSupervisado1 = dfNoSupervisado1.reset_index(drop=True)
N = dfNoSupervisado1.shape[0]

catColumnsPos = [dfNoSupervisado1.columns.get_loc(col) for col in list(dfNoSupervisado1.select_dtypes('object').columns)]
numColumnsPos = [dfNoSupervisado1.columns.get_loc(col) for col in list(dfNoSupervisado1.select_dtypes('float64').columns)]

dataNum=dfNoSupervisado1.iloc[:,numColumnsPos]
dataNum=dataNum.to_numpy()
print('Datos numéricos',type(dataNum))

dataCat=dfNoSupervisado1.iloc[:,catColumnsPos]
dataCat=dataCat.to_numpy()
print('Datos categóricos',type(dataCat))

# gamma = 0.5 (constante)
catWeight=0.5

# Inicializamos la matriz de distancias con ceros
scores = [ [ 0 for i in range(N) ] for j in range(N) ]

# Calculamos la mitad de la matriz distancias
for i in range(N):
  for j in range(i+1,N):
    scores[i][j]=np.sum((dataNum[i] - dataNum[j]) ** 2) + catWeight * np.sum(dataCat[i]!=dataCat[j])

# Completamos la matriz
for i in range(0,N):
  for j in range(0,i):
    scores[i][j]=scores[j][i]

Datos numéricos <class 'numpy.ndarray'>
Datos categóricos <class 'numpy.ndarray'>
Inicio --- 3.2901763916015625e-05 seconds ---


In [ ]:
cost = []
silueta = []
start_time = time.time()

for k in (range(1,11)):
# try:
    kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = 0.5, random_state = 0)
    kprototype.fit_predict(dfNoSupervisado1, categorical = catColumnsPos)
    cost.append(kprototype.cost_)
    print('k: {}'.format(k))
    print("--- %s seconds ---" % (time.time() - start_time))
    if k != 1:
        silhouette_coeff = silhouette_score(scores, kprototype.labels_, metric='precomputed')
        silueta.append(silhouette_coeff)
        print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
        print(silhouette_coeff)
    # except:
    #     break

In [ ]:
silueta

In [ ]:
df_cost = pd.DataFrame({'Cluster':range(1,11), 'Cost':cost})
df_silueta = pd.DataFrame({'Cluster':range(2,11), 'Coeficiente':silueta})

In [ ]:
# Codo
plotnine.options.figure_size = (8, 4.8)
(
ggplot(data = df_cost)+
geom_line(aes(x = 'Cluster',
y = 'Cost'))+
geom_point(aes(x = 'Cluster',
y = 'Cost'))+
geom_label(aes(x = 'Cluster',
y = 'Cost',
label = 'Cluster'),
size = 10,
nudge_y = 1000) +
labs(title = 'Optimal number of cluster with Elbow Method')+
xlab('Number of Clusters k')+
ylab('Cost')+
theme_minimal()
)

In [ ]:
# Coeficiente silueta
plotnine.options.figure_size = (8, 4.8)
(
ggplot(data = df_silueta)+
geom_line(aes(x = 'Cluster',
y = 'Coeficiente'))+
geom_point(aes(x = 'Cluster',
y = 'Coeficiente'))+
geom_label(aes(x = 'Cluster',
y = 'Coeficiente',
label = 'Cluster'),
size = 10,
nudge_y = 0.1) +
labs(title = 'Optimal number of cluster with Silhouette coefficient')+
xlab('Number of Clusters k')+
ylab('Coeficiente')+
theme_minimal()
)

In [ ]:
# Fit the cluster 
k = 2
gamma = 0.5
import time
start_time = time.time()
kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototype.fit_predict(dfNoSupervisado1, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
dfNoSupervisado1['Cluster'] = kprototype.labels_

In [ ]:
import matplotlib.pylab as plt
for var in ['COSTX','QTYVAR']:
    for i in range(k):
        sns.distplot(dfNoSupervisado1.loc[dfNoSupervisado1['Cluster']==i,var], label= 'Cluster {}'.format(i));
        plt.legend(loc=(1.04,0))
    plt.show();

In [ ]:
for i in range(k):
    sns.distplot(dfNoSupervisado1.loc[dfNoSupervisado1['Cluster']==i,'QTYVAR'], label= 'Cluster {}'.format(i));
    plt.legend(loc=(1.04,0))
    plt.show()